In [5]:
##PROMPT##

##Прочитай текст следующей главы и сделай для нее краткое содержание на русском. 
##Краткое содержание должно включать заголовок и номер главы; события главы в 2-3 абзацах (заголовок События главы); 
##список персонажей, участвующих в главе, с описанием их внешнего вида - возраст, кожа, волосы, глаза, рост, одежда, поза и так далее, если указано (Персонажи главы); 
##список локаций главы и их описание, если указано (Локации главы); погода и время дня, если указано (Погода и время дня). Избегай предположений. Use temperature = 0.3##




import ipywidgets as widgets
import pandas as pd
import io
import re
import os
import pythoncom
import math
from win32com import client as win32

from IPython.display import display
from docx import Document
from collections import defaultdict
from words import (
    clothes_words, hair_words, appearances_words, weather_words,
    locations_words, age_words, other_words
)


# Создаем виджет для загрузки файла
upload_widget = widgets.FileUpload(
    accept = '.docx',  # Принимаем только файлы .docx
    multiple = False  # Только один файл
)

# Создаем кнопки
analyze_button = widgets.Button(
    description = 'Check',
    disabled = False,
    button_style = '',  # 'success', 'info', 'warning', 'danger' или ''
    tooltip = 'Check',
    icon = ''  # Иконка (имя FontAwesome без префикса `fa-`)
)

split_by_chapters_button = widgets.Button(
    description = 'Split',
    disabled = False,
    button_style = '',  # 'success', 'info', 'warning', 'danger' или ''
    tooltip = 'Split',
    icon = ''  # Иконка (имя FontAwesome без префикса `fa-`)
)

find_appearance_button = widgets.Button(
    description = 'Find Appearance',
    disabled = False,
    button_style = '',  # 'success', 'info', 'warning', 'danger' или ''
    tooltip = 'Find Appearance',
    icon = ''  # Иконка (имя FontAwesome без префикса `fa-`)
)

clean_summary_button = widgets.Button(
    description = 'Clean summary',
    disabled = False,
    button_style = '',  # 'success', 'info', 'warning', 'danger' или ''
    tooltip = 'Clean summary',
    icon = ''  # Иконка (имя FontAwesome без префикса `fa-`)
)

cover_info_button = widgets.Button(
    description = 'Create cover file',
    disabled = False,
    button_style = '',  # 'success', 'info', 'warning', 'danger' или ''
    tooltip = 'Create cover file',
    icon = ''  # Иконка (имя FontAwesome без префикса `fa-`)
)

# Создаем виджет вывода
output = widgets.Output()


def convert_docm_to_docx(input_path, output_path):
    """Конвертирует файл .docm в .docx с помощью Microsoft Word."""
    try:
        pythoncom.CoInitialize()
        word = win32.Dispatch("Word.Application")
        word.Visible = False  # Запуск Word в фоновом режиме

        print(f"Конвертация {input_path} в {output_path}...")
        doc = word.Documents.Open(input_path)
        doc.SaveAs(output_path, FileFormat=16)  # 16 — это формат для .docx
        doc.Close()
        word.Quit()
        print(f"Файл успешно конвертирован: {output_path}")
        return True
    except Exception as e:
        print(f"Ошибка при конвертации: {e}")
        return False
    finally:
        pythoncom.CoUninitialize()


def is_valid_docx(file_content):
    """Проверяет, можно ли открыть файл как документ Word."""
    try:
        docx_file = io.BytesIO(file_content)
        Document(docx_file)  # Попытка открыть как .docx
        return True
    except Exception:
        return False

def load_docx_file(file_info):
    try:
        filename = file_info['name']
        file_content = file_info['content']

        if not is_valid_docx(file_content):
            print(f"Ошибка: Файл {filename} повреждён или не является корректным .docx/.docm.")
            return None  # Добавляем return здесь, чтобы завершить выполнение

        # Проверяем расширение и обрабатываем файл .docm
        if filename.endswith('.docm'):
            print("Обнаружен .docm файл, выполняется конвертация...")
            temp_docm_path = "temp_file.docm"
            with open(temp_docm_path, 'wb') as f:
                f.write(file_content)

            temp_docx_path = "temp_file.docx"
            if convert_docm_to_docx(temp_docm_path, temp_docx_path):
                print("Загрузка сконвертированного .docx файла...")
                with open(temp_docx_path, 'rb') as f:
                    docx_file = io.BytesIO(f.read())

                # Удаление временных файлов
                os.remove(temp_docm_path)
                os.remove(temp_docx_path)
            else:
                print("Конвертация не удалась.")
                return None
        else:
            print("Загружается файл .docx...")
            docx_file = io.BytesIO(file_content)

        # Пытаемся открыть документ Word
        docx = Document(docx_file)
        print("Файл успешно загружен и прочитан.")
        return docx
    except Exception as e:
        print(f"Ошибка при загрузке файла: {e}")
        return None



# Функции для обработки нажатия кнопок
def on_analyze_button_click(event):
    with output:
        output.clear_output()  # Очищаем предыдущий вывод
        if upload_widget.value:
            # Получаем первый элемент кортежа (информацию о загруженном файле)
            file_info = list(upload_widget.value)[0]
            filename = file_info['name']  # Извлекаем имя файла
            print(f'Загруженный файл: {filename}')
            
            docx_doc = load_docx_file(file_info)
            analyze_book(docx_doc)
        else:
            print("Файл не загружен")

def on_split_by_chapters_button_click(event):
    with output:
        output.clear_output()  # Очищаем предыдущий вывод
        if upload_widget.value:
            # Получаем первый элемент кортежа (информацию о загруженном файле)
            file_info = list(upload_widget.value)[0]
            filename = file_info['name']  # Извлекаем имя файла
            
            print(f'Загруженный файл: {filename}')

            path = os.path.splitext(filename)[0]
            docx_doc = load_docx_file(file_info)
            
            split_chapters(docx_doc, path)
        else:
            print("Файл не загружен")

def on_find_appearance_button_click(event):
    with output:
        output.clear_output()  # Очищаем предыдущий вывод
        if upload_widget.value:
            # Получаем первый элемент кортежа (информацию о загруженном файле)
            file_info = list(upload_widget.value)[0]
            filename = file_info['name']  # Извлекаем имя файла
            
            print(f'Загруженный файл: {filename}')

            path = os.path.splitext(filename)[0]
            docx_doc = load_docx_file(file_info)
            
            find_appearance(docx_doc, path)
        else:
            print("Файл не загружен")

def on_clean_summary_button_click(event):
    with output:
        output.clear_output()  # Очищаем предыдущий вывод
        if upload_widget.value:
            # Получаем первый элемент кортежа (информацию о загруженном файле)
            file_info = list(upload_widget.value)[0]
            filename = file_info['name']  # Извлекаем имя файла
            
            print(f'Загруженный файл: {filename}')

            path = os.path.splitext(filename)[0]
            docx_doc = load_docx_file(file_info)
            
            clean_summary(docx_doc, path)
        else:
            print("Файл не загружен")

def on_cover_info_button_click(event):
    with output:
        output.clear_output()  # Очищаем предыдущий вывод
        if upload_widget.value:
            # Получаем первый элемент кортежа (информацию о загруженном файле)
            file_info = list(upload_widget.value)[0]
            filename = file_info['name']  # Извлекаем имя файла
            
            print(f'Загруженный файл: {filename}')

            path = os.path.splitext(filename)[0]
            docx_doc = load_docx_file(file_info)
            
            create_cover_info(docx_doc, path)
        else:
            print("Файл не загружен")
            

# Привязываем функции к кнопке
analyze_button.on_click(on_analyze_button_click)
split_by_chapters_button.on_click(on_split_by_chapters_button_click)
find_appearance_button.on_click(on_find_appearance_button_click)
clean_summary_button.on_click(on_clean_summary_button_click)
cover_info_button.on_click(on_cover_info_button_click)

# Отображаем виджеты
display(upload_widget, analyze_button, split_by_chapters_button, find_appearance_button, clean_summary_button, cover_info_button, output)

# Словарь для преобразования чисел из текстового формата
TEXT_NUMBERS = {
    'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6,
    'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10, 'eleven': 11, 'twelve': 12,
    'thirteen': 13, 'fourteen': 14, 'fifteen': 15, 'sixteen': 16,
    'seventeen': 17, 'eighteen': 18, 'nineteen': 19, 'twenty': 20,
    'thirty': 30, 'forty': 40, 'fifty': 50, 'sixty': 60, 'seventy': 70,
    'eighty': 80, 'ninety': 90, 'hundred': 100
}


    


### 00_book_Check ###

def word_to_number(word):
    """Преобразует текстовое число в числовое значение."""
    words = re.split(r'[\s\-]+', word.lower())  # Разбиваем по пробелам и дефисам
    number = 0
    temp = 0

    for w in words:
        if w in TEXT_NUMBERS:
            scale = TEXT_NUMBERS[w]
            if scale == 100:  # Обработка сотен, например "one hundred"
                temp *= scale
            else:
                temp += scale
        else:
            return None  # Если слово не распознано, возвращаем None

    number += temp
    return number

def extract_chapter_number(text):
    """Извлекает номер главы из текста."""
    # Проверяем числовой формат, например, "Chapter 1" или "Chapter 1 - School Trouble"
    numeric_match = re.match(r'Chapter\s+(\d+)', text, re.IGNORECASE)
    if numeric_match:
        return int(numeric_match.group(1))
    
    # Проверяем текстовый формат, например, "Chapter One" или "Chapter Thirty-one"
    text_match = re.match(r'Chapter\s+([\w\s\-]+)', text, re.IGNORECASE)
    if text_match:
        word = text_match.group(1).strip()
        return word_to_number(word)
    return None

def extract_chapters(doc):
    """Извлекает главы и их содержимое из .docx файла."""
    chapters = []
    current_chapter = None
    current_text = []

    for para in doc.paragraphs:
        chapter_num = extract_chapter_number(para.text.strip())
        if chapter_num is not None:
            if current_chapter is not None and current_text:
                chapters.append((current_chapter, "\n".join(current_text).strip()))
            current_chapter = chapter_num
            current_text = []
        else:
            current_text.append(para.text.strip())

    if current_chapter is not None and current_text:
        chapters.append((current_chapter, "\n".join(current_text).strip()))

    return chapters

def check_duplicate_chapters(chapters):
    chapter_numbers = [chap[0] for chap in chapters]
    return {chap for chap in chapter_numbers if chapter_numbers.count(chap) > 1}

def check_text_duplicates(chapters):
    """Проверяет дублирующийся текст в главах с учётом нормализации."""
    text_to_chapter = defaultdict(list)

    for chapter, text in chapters:
        # Нормализуем текст: убираем лишние пробелы и приводим к нижнему регистру
        normalized_text = " ".join(text.split()).lower()
        
        # Добавляем номер главы в список глав с таким же текстом
        text_to_chapter[normalized_text].append(chapter)

    # Фильтруем только те записи, где один и тот же текст встречается более одного раза
    return {text: chaps for text, chaps in text_to_chapter.items() if len(chaps) > 1}

def check_chapter_order(chapters):
    chapter_numbers = [chap[0] for chap in chapters]
    return [(chapter_numbers[i], chapter_numbers[i+1]) 
            for i in range(len(chapter_numbers) - 1) if chapter_numbers[i] > chapter_numbers[i + 1]]

def check_missing_chapters(chapters):
    """Проверяет пропущенные главы в последовательности."""
    chapter_numbers = sorted([chap[0] for chap in chapters])
    missing_chapters = [num for num in range(chapter_numbers[0], chapter_numbers[-1] + 1) 
                        if num not in chapter_numbers]
    return missing_chapters

def analyze_book(docx_file):
    chapters = extract_chapters(docx_file)

    print("Список найденных глав:")
    for chapter, text in chapters:
        print(f"Chapter {chapter}: {text[:50]}...")
    
    duplicate_chapters = check_duplicate_chapters(chapters)
    duplicate_texts = check_text_duplicates(chapters)
    unordered_chapters = check_chapter_order(chapters)
    missing_chapters = check_missing_chapters(chapters)

    print("\nРезультаты проверок:")
    print(f"❌ Найдены повторяющиеся главы: {duplicate_chapters}" if duplicate_chapters else "✅ Повторяющиеся главы не найдены.")
    print(f"❌ Найдены главы с дублирующимся текстом: {duplicate_texts}" if duplicate_texts else "✅ Дублирующийся текст не найден.")
    print(f"❌ Найдены главы с нарушением порядка: {unordered_chapters}" if unordered_chapters else "✅ Нарушений в порядке глав не обнаружено.")
    print(f"❌ Пропущенные главы: {missing_chapters}" if missing_chapters else "✅ Пропущенных глав не найдено.")



### 01_Breaking_into_chapters ###

def split_chapters(doc, path):
    #print('split_chapters(doc_path in doc.paragraph)')
    # Буфер для накопления содержимого текущей главы
    current_chapter = []
    chapter_number = 0

    # Регулярное выражение для поиска заголовков глав, включая дефис
    chapter_pattern = re.compile('Chapter', re.IGNORECASE)
    found_first_chapter = False  # Флаг для пропуска текста до первой главы

    # Проходим по параграфам документа
    for paragraph in doc.paragraphs:
        # Если параграф соответствует заголовку главы
        if chapter_pattern.match(paragraph.text.strip()):
            if found_first_chapter and current_chapter:
                save_chapter(current_chapter, chapter_number, path)

            # Очищаем буфер для новой главы и увеличиваем номер главы
            current_chapter = [paragraph.text]
            chapter_number += 1
            found_first_chapter = True  # Первая глава найдена
        else:
            # Добавляем содержимое параграфа в текущую главу, только если первая глава уже найдена
            if found_first_chapter:
                current_chapter.append(paragraph.text)

    # Сохраняем последнюю главу
    if current_chapter:
        save_chapter(current_chapter, chapter_number, path)

def save_chapter(chapter_content, chapter_number, path):
    print('save')
    # Создаем новый документ для главы
    new_doc = Document()
    
    # Добавляем содержимое главы в новый документ
    for paragraph in chapter_content:
        new_doc.add_paragraph(paragraph)
    
    # Указываем папку для сохранения
    save_directory = f'{path}'  # замените на нужный путь
    
    # Убедимся, что папка существует, если нет — создаем
    os.makedirs(save_directory, exist_ok = True)
    
    # Сохраняем документ с названием Chapter_N.docx в указанную папку
    chapter_filename = os.path.join(save_directory, f'Chapter_{chapter_number}.docx')
    new_doc.save(chapter_filename)
    print(f'Chapter {chapter_number} saved as {chapter_filename}')



### 02_looking_for_appearance ###

def search_words_in_chapter(chapter_text, words):
    """Ищет слова в тексте главы и возвращает их с позициями."""
    word_pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    matches = []

    for match in re.finditer(word_pattern, chapter_text, re.IGNORECASE):
        start, end = match.start(), match.end()

        # Собираем контексты вокруг найденного слова
        start_context = chapter_text[:start].split()[-8:]
        end_context = chapter_text[end:].split()[:8]
        found_word = chapter_text[start:end]
        result = ' '.join(start_context + [found_word] + end_context)

        # Сохраняем позицию и результат
        matches.append((start, result))

    return matches

def search_in_all_chapters(chapters, words, category):
    """Ищет слова во всех главах и подсчитывает общее количество совпадений."""
    total_matches = 0
    results = []

    for chapter_number, chapter_title, chapter_text in chapters:
        matches = search_words_in_chapter(chapter_text, words)
        total_matches += len(matches)

        for position, match in matches:
            results.append([chapter_number, chapter_title, position, match, category])

    print(f"✅ Категория '{category}': найдено {total_matches} совпадений.")
    return results

def split_text_into_chapters(book_text):
    """Разделяет текст на главы. Пример: разделение по слову 'Chapter'."""
    chapters = []
    chapter_texts = re.split(r'Chapter', book_text, flags=re.IGNORECASE)
    for i, text in enumerate(chapter_texts[1:], start=1):
        title_end = text.find('\n')
        chapter_title = text[:title_end].strip()
        chapter_content = text[title_end + 1:].strip()
        chapters.append((i, chapter_title, chapter_content))
    return chapters

def find_appearance(doc, path):

    print("📖 Извлечение текста из файла...")

    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
        
    book_text = '\n'.join(full_text)

    print("✂️ Разделение текста на главы...")
    chapters = split_text_into_chapters(book_text)
    if not chapters:
        print("❌ Не удалось разделить текст на главы.")
        return


    all_results = []
    for words, category in [
        (clothes_words, 'Clothes'), (hair_words, 'Hair'),
        (appearances_words, 'Appearances'), (weather_words, 'Weather'),
        (locations_words, 'Locations'), (age_words, 'Age'), (other_words, 'Other')
    ]:
        all_results.extend(search_in_all_chapters(chapters, words, category))

    print("📊 Создание DataFrame с результатами...")
    all_data_df = pd.DataFrame(all_results, columns=['Chapter Number', 'Chapter Title', 'Position', 'Match', 'Category'])
    
    # Сортируем по номеру главы и позиции
    all_data_df.sort_values(by = ['Chapter Number', 'Position'], inplace = True)
    # Удаляем колонку с позициями слова
    all_data_df = all_data_df.drop(columns=['Position'])

    print("💾 Сохранение результатов в Excel...")
    os.makedirs(path, exist_ok = True)
    excel_file_path = f"{path}/Details.xlsx"

    try:
        all_data_df.to_excel(excel_file_path, sheet_name='Appearance', index = False)
        print(f"✅ Файл Excel создан: {excel_file_path}")
    except Exception as e:
        print(f"❌ Ошибка при сохранении Excel файла: {e}")

##03_clean_summary##
def remove_inserts_and_format_headings(file_path, save_path):
    # Открываем документ
    doc = Document(file_path)

    # Регулярные выражения для поиска вставок, которые нужно удалить
    patterns = [
        r'\d+o',  # например, '4o'
        r'Вы сказали:',  # например, 'Вы сказали:'
        r'Chapter_\d+\.docx',  # например, 'Chapter_2.docx', 'Chapter_3.docx'
        r'Документ',  # например, 'Документ'
        r'ChatGPT',  # например, 'ChatGPT'
        r'^Этот контент может нарушать нашу политику использования\.$',  # точное совпадение фразы
        r'^Память обновлена$'  # точное совпадение фразы
    ]

    # Регулярное выражение для заголовков глав: Краткое содержание главы + номер
    chapter_heading_pattern = re.compile(r'^Глава\s+\d+')

    # Функция для проверки, нужно ли удалять абзац
    def should_delete(paragraph):
        for pattern in patterns:
            if re.match(pattern, paragraph.text.strip()):  # Только если абзац целиком совпадает с шаблоном
                return True
        return False

    # Создаем новый документ для сохранения изменений
    new_doc = Document()

    # Проходим по абзацам и удаляем те, которые соответствуют шаблонам
    for para in doc.paragraphs:
        if not should_delete(para):
            # Если абзац является заголовком главы, применяем стиль "Заголовок 3"
            if chapter_heading_pattern.match(para.text.strip()):
                new_doc.add_paragraph(para.text, style='Heading 3')
            else:
                # Добавляем абзац в новый документ, сохраняя форматирование
                new_para = new_doc.add_paragraph()
                for run in para.runs:
                    new_run = new_para.add_run(run.text)
                    # Сохраняем форматирование
                    new_run.bold = run.bold
                    new_run.italic = run.italic
                    new_run.underline = run.underline
                    new_run.font.name = run.font.name
                    new_run.font.size = run.font.size

    # Сохраняем новый документ
    new_doc.save(save_path)

    # Уведомление о сохранении файла
    abs_path = os.path.abspath(save_path)
    print(f"Файл успешно сохранен по адресу: {abs_path}")

# Пример использования
def clean_summary(docx_doc, path):
    file_path = f"{path}/summary.docx"  # Исходный файл
    save_path = f"{path}/summary_clean.docx"  # Файл для сохранения без вставок
    
    remove_inserts_and_format_headings(file_path, save_path)



##04_Cover##
def extract_sections(doc, summary_label, location_label, character_label):
    summaries = []
    locations = []
    characters = []

    current_section = ""
    section_type = None

    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()

        if summary_label in text:
            if current_section:
                if section_type == "location":
                    locations.append(current_section.strip())
                elif section_type == "character":
                    characters.append(current_section.strip())
            section_type = "summary"
            current_section = ""

        elif location_label in text:
            if current_section:
                if section_type == "summary":
                    summaries.append(current_section.strip())
                elif section_type == "character":
                    characters.append(current_section.strip())
            section_type = "location"
            current_section = ""

        elif character_label in text:
            if current_section:
                if section_type == "summary":
                    summaries.append(current_section.strip())
                elif section_type == "location":
                    locations.append(current_section.strip())
            section_type = "character"
            current_section = ""

        elif text == "":
            continue

        if section_type == "summary":
            current_section += text + " "
        elif section_type == "location":
            current_section += text + " "
        elif section_type == "character":
            current_section += text + " "

    if current_section:
        if section_type == "summary":
            summaries.append(current_section.strip())
        elif section_type == "location":
            locations.append(current_section.strip())
        elif section_type == "character":
            characters.append(current_section.strip())

    return summaries, locations, characters

# Функция для сохранения данных в файлы
def save_to_docx(data, output_path, title_prefix):
    doc = Document()
    for idx, section in enumerate(data, start=1):
        doc.add_heading(f"{title_prefix} {idx}", level=1)
        doc.add_paragraph(section)
    doc.save(output_path)
    print(f"Сохранено: {output_path}")

# Функция для деления summaries на 3 части, не разрывая главы
def split_summaries_evenly(summaries):
    chunk_size = math.ceil(len(summaries) / 3)
    chunks = []

    start = 0
    for i in range(3):
        end = min(start + chunk_size, len(summaries))
        chunks.append(summaries[start:end])
        start = end

    return chunks
    
def create_cover_info(docx_doc, path):
    summary_label = "События главы"
    location_label = "Локации главы"
    character_label = "Персонажи главы"

    output_path = path
    summary = Document(f"{output_path}/summary_clean.docx")
    summaries, locations, characters = extract_sections(summary, summary_label, location_label, character_label)
    # Сохраняем основные файлы
    save_to_docx(summaries, f"{output_path}/synopsis.docx", "События главы")
    save_to_docx(locations, f"{output_path}/locations.docx", "Локации главы")
    save_to_docx(characters, f"{output_path}/characters.docx", "Персонажи главы")

    # Делим summaries на 3 части и сохраняем их
    chunks = split_summaries_evenly(summaries)
    for i, chunk in enumerate(chunks, start=1):
        save_to_docx(chunk, f"{output_path}/summary_part_{i}.docx", "Глава")

    print("Файлы успешно сохранены!")
    
    


FileUpload(value=(), accept='.docx', description='Upload')

Button(description='Check', style=ButtonStyle(), tooltip='Check')

Button(description='Split', style=ButtonStyle(), tooltip='Split')

Button(description='Find Appearance', style=ButtonStyle(), tooltip='Find Appearance')

Button(description='Clean summary', style=ButtonStyle(), tooltip='Clean summary')

Button(description='Create cover file', style=ButtonStyle(), tooltip='Create cover file')

Output()